In [8]:
import time
import sqlite3
import pandas as pd
import sqlalchemy, os
from datetime import datetime
from dotenv import load_dotenv
from binance.client import Client

load_dotenv()

API_KEY = os.environ.get("API_KEY")
API_SECRET = os.environ.get("API_SECRET")
client = Client(API_KEY, API_SECRET)

db = sqlite3.connect('crypto.db')
db.cursor().execute("""
    CREATE TABLE inventory (
        symbol TEXT,
        avg_price REAL,
        qty REAL,
        buy_time REAL
    )""")
db.commit()
db.close()

In [9]:
def fetchall():
    db = sqlite3.connect('crypto.db')
    cursor = db.cursor()
    cursor.execute("SELECT * FROM inventory")
    alldata = cursor.fetchall()
    db.commit()
    db.close()
    return alldata

def extract_from_tuple(alldata, i = 0):
    symbols = [x[i] for x in alldata]
    return symbols

def execute(command):
    db = sqlite3.connect('crypto.db')
    cursor = db.cursor()
    cursor.execute(command)
    db.commit()
    db.close()
    
def add_to_db(symbol, price):
    time = int(datetime.utcnow().strftime('%d%H%M%S'))
    cursor.execute(f"INSERT INTO inventory VALUES ('{symbol}',{price},{time})")
    
def delete_from_db(symbol):
    cursor.execute(f"DELETE FROM inventory WHERE symbol = '{symbol}'")

def get_data(symbol):
    db = sqlite3.connect('crypto.db')
    cursor = db.cursor()
    cursor.execute(f"SELECT * FROM inventory WHERE symbol = '{symbol}'")
    data = cursor.fetchone()
    db.commit()
    db.close()
    return data

def buy_symbol(symbol):
    qty=1
    price = float(client.get_avg_price(symbol=symbol)['price'])
    add_to_db(symbol,price,qty)
    print(f"{symbol} added to inventory at ${price}")
    # order = client.create_order(symbol="SLPUSDT", side="BUY", type="MARKET", quantity=qty)
    # print(order)

def sell_symbol(symbol):
    qty=get_data(symbol)[2]
    price = float(client.get_avg_price(symbol=symbol)['price'])
    buy_price = get_data(symbol)[1]
    profit = float("{:.4f}".format(((price-buy_price)/buy_price)*100))
    # order = client.create_order(symbol="SLPUSDT", side="SELL", type="MARKET", quantity=qty)
    # print(order)
    delete_from_db(symbol)
    print(f"Sold {symbol} at ${price} with {profit}% profit")

In [ ]:
def strategy(list_of_symbols):
    inventory = extract_from_tuple(fetchall())
    for symbol in list_of_symbols:
        if symbol not in inventory:
            buy_symbol(symbol)
            
    for symbol in inventory:
        buy_time = get_data(symbol)[3]
        time = int(datetime.utcnow().strftime('%d%H%M%S'))
        # If more than 15 minutes have passed
        if(time >= buy_time + 1500):
            sell_symbol(symbol)

def start():
    with open('pairs_level_4.txt','r') as file:
        file.seek(0) # Ensure you're at the start of the file..
        first_char = file.read(1) # Get the first character
        if not first_char:
            pass # The first character is the empty string..
        else:
            file.seek(0) # The first character wasn't empty. Return to the start of the file.
             # Use file now
            strategy(file.readline().split(','))

print("Program starts")

while(True):
    time.sleep(5)
    start()

Program starts
